In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors
from pyspark.sql import SQLContext
from pyspark.ml.param import Param, Params
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import udf
from IPython.display import display

import sys
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import os.path

In [ ]:
textFile = sc.textFile("dataset2016.csv")

In [ ]:
textRDD = file.map(lambda x: x.split(','))
header = textRDD.first()
finalRDD = textRDD.filter(lambda r: r != header)

airline_data = sqlContext.createDataFrame(finalRDD)

airlinedata = airline_data.toDF("Year", "Month", "DayOfMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest", "CRSDepTime", "DepTime", "DepDelay" ,"CRSArrTime", "ArrTime", "Distance", "CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "EMPTY")

In [ ]:
airlinedata = airlinedata.withColumn("Year", airlinedata["Year"].cast("int"))
airlinedata = airlinedata.withColumn("Month", airlinedata["Month"].cast("int"))
airlinedata = airlinedata.withColumn("DayOfMonth", airlinedata["DayOfMonth"].cast("int"))
airlinedata = airlinedata.withColumn("DayOfWeek", airlinedata["DayOfWeek"].cast("int"))
airlinedata = airlinedata.withColumn("DepDelay", airlinedata["DepDelay"].cast("float"))
airlinedata = airlinedata.withColumn("Distance", airlinedata["Distance"].cast("float"))
airlinedata = airlinedata.withColumn("CarrierDelay", airlinedata["CarrierDelay"].cast("float"))
airlinedata = airlinedata.withColumn("WeatherDelay", airlinedata["WeatherDelay"].cast("float"))
airlinedata = airlinedata.withColumn("NASDelay", airlinedata["NASDelay"].cast("float"))
airlinedata = airlinedata.withColumn("SecurityDelay", airlinedata["SecurityDelay"].cast("float"))
airlinedata = airlinedata.withColumn("LateAircraftDelay", airlinedata["LateAircraftDelay"].cast("float"))

In [ ]:
airlinedata = airlinedata.withColumn('DepDelayed', airlinedata['DepDelay']>15)

airlinedata.registerTempTable("airlinedata")

In [ ]:
delay_causes = sqlContext.sql("SELECT sum(WeatherDelay) Weather,sum(NASDelay) NAS,sum(SecurityDelay) Security,sum(LateAircraftDelay) lateAircraft,sum(CarrierDelay) Carrier\
                              FROM airlinedata ")
file_delay_causes = delay_causes.toPandas()
file_delay_causes.head()

In [ ]:
group_delay = sqlContext.sql("SELECT Origin, count(*) conFlight,avg(DepDelay) delay \
                                FROM airlinedata \
                                GROUP BY Origin")

origin = group_delay.toPandas()
origin.sort('delay',ascending=0).head()

In [ ]:
first_file = sc.file("airports.dat")
finalRDD1 = first_file.map(lambda x: x.split(','))
first_df = sqlContext.createDataFrame(finalRDD1)
df = first_df.toDF("SerialNo", "Name", "City", "Country", "IATA", "ICAO", "lat", "lng", "alt", "Tzone" ,"DST", "Tz", "A", "B")
df = df.withColumn("SerialNo", df["SerialNo"].cast("int"))
df = df.withColumn("lat", df["lat"].cast("double"))
df = df.withColumn("lng", df["lng"].cast("double"))
df = df.withColumn("alt", df["alt"].cast("int"))
df = df.withColumn("Tzone", df["Tzone"].cast("int"))
## It shows us that airport has been matched with latitude and longitude
df.take(5)

In [ ]:
second_file = sc.file("/FileStore/tables/5ruz1dx81492859916686/AmericaAirports.csv")
finalRDD2 = second_file.map(lambda x: x.split(','))
df_2 = sqlContext.createDataFrame(finalRDD2)
df_air = df_2.toDF("SerialNo", "Name", "City", "Country", "IATA", "ICAO", "lat", "lng", "alt", "Tzone" ,"DST", "Tz", "A")
df_air = df_air.withColumn("SerialNo", df_air["SerialNo"].cast("int"))
df_air = df_air.withColumn("lat", df_air["lat"].cast("double"))
df_air = df_air.withColumn("lng", df_air["lng"].cast("double"))
df_air = df_air.withColumn("alt", df_air["alt"].cast("int"))
df_air = df_air.withColumn("Tzone", df_air["Tzone"].cast("int"))

In [ ]:
df_aira = df_air.toPandas()
origin['Origin'] = origin['Origin'].apply(lambda s:s.replace('"', ""))

df_airports = pd.merge(origin, df_aira, left_on = 'Origin', right_on = 'IATA')
df_airports.sort('delay',ascending=0).head()

In [ ]:
df_airports.to_csv(sys.stdout)

In [ ]:
route_delay_group = sqlContext.sql("SELECT Origin, Dest, count(*) traffic,avg(Distance) avgDist,\
                                    avg(DepDelay) avgDelay\
                                FROM airlinedata \
                                GROUP BY Origin,Dest")
route_with_delay = route_delay_group.toPandas()

route_with_delay['Origin'] = route_with_delay['Origin'].apply(lambda s:s.replace('"', ""))
route_with_delay['Dest'] = route_with_delay['Dest'].apply(lambda s:s.replace('"', ""))

df_airport_rout1 = pd.merge(route_with_delay, df_aira, left_on = 'Origin', right_on = 'IATA')

df_airport_rout2 = pd.merge(df_airport_rout1, df_aira, left_on = 'Dest', right_on = 'IATA')

df_airport_rout = df_airport_rout2[["Origin","lat_x","lng_x","Dest","lat_y","lng_y",\
                                    "avgDelay", "traffic"]]

df_airport_rout.sort('avgDelay',ascending=0).head()

df_airport_rout.to_csv(sys.stdout)

In [ ]:
Origin_Airport="\"DFW\""

df_ORG = sqlContext.sql("SELECT * from airlinedata WHERE Origin='"+Origin_Airport+"'")

df_ORG.registerTempTable("df_ORG")
df_ORG.select('DayOfWeek','DayofMonth','DepDelay','Origin','Dest','CRSDepTime').show(2)

print "total flights from this airport: " + str(df_ORG.count())

grp_carr = sqlContext.sql("SELECT  UniqueCarrier,DayOfMonth,avg(DepDelay) avgDelay from df_ORG \
                            WHERE DepDelayed=True \
                            GROUP BY UniqueCarrier,DayOfMonth")
s = grp_carr.toPandas()